In [ ]:
# !pip install pyspark
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from datetime import datetime
import seaborn as sns
from collections import Counter
from itertools import combinations

try:
    from mlxtend.frequent_patterns import apriori, association_rules
    from mlxtend.preprocessing import TransactionEncoder
    MLXTEND_AVAILABLE = True
except ImportError:
    print("mlxtend not available. Install with: pip install mlxtend")
    MLXTEND_AVAILABLE = False
############################### FIX IMPORT #####################################

In [ ]:
n = 10

In [ ]:
dataFull = pd.read_csv('/logon.csv')
print("Raw schema: \n",dataFull.head())

# Drop the columns
if 'file_tree' in dataFull.columns or 'Unnamed: 0' in dataFull.columns:
    data = dataFull.drop(columns=[col for col in ['file_tree', 'Unnamed: 0'] if col in dataFull.columns])
    print("Final schema: \n",data.head())
else:
  data = dataFull
print('\n-----------------------------------------------------------------------------\n')

# How many instances have null values if any
if(data.isnull().values.any()):
  data.isnull().sum()
  print('\n-----------------------------------------------------------------------------\n')

# unique values in columns
print(data.nunique())
print('\n-----------------------------------------------------------------------------\n')

Raw schema: 
                          id                 date     user       pc activity
0  {X1D9-S0ES98JV-5357PWMI}  01/02/2010 06:49:00  NGF0157  PC-6056    Logon
1  {G2B3-L6EJ61GT-2222RKSO}  01/02/2010 06:50:00  LRR0148  PC-4275    Logon
2  {U6Q3-U0WE70UA-3770UREL}  01/02/2010 06:53:04  LRR0148  PC-4124    Logon
3  {I0N5-R7NA26TG-6263KNGM}  01/02/2010 07:00:00  IRM0931  PC-7188    Logon
4  {D1S0-N6FH62BT-5398KANK}  01/02/2010 07:00:00  MOH0273  PC-6699    Logon

-----------------------------------------------------------------------------

id          854859
date        338041
user          1000
pc            1003
activity         2
dtype: int64

-----------------------------------------------------------------------------



In [ ]:
# Suppress specific deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Create DataFrame from data
columns = ['id', 'date', 'user', 'pc', 'activity']
df = pd.DataFrame(data, columns=columns)

print(f"Original data shape: {df.shape}")

# FREQUENT PATTERN MINING
print(f"Dataset size: {len(df):,} records") #comma to add separator at every thousand
print(f"Unique items in transactions: Users({df['user'].nunique():,}), PCs({df['pc'].nunique():,})")


Original data shape: (854859, 5)
Dataset size: 854,859 records
Unique items in transactions: Users(1,000), PCs(1,003)


In [ ]:
# LOGIN/LOGOUT ANALYSIS USING BUILT-IN FUNCTIONS
# Count logins per user using value_counts
login_data = df[df['activity'] == 'Logon']
user_logins = login_data['user'].value_counts().reset_index()
user_logins.columns = ['user', 'login_count']

print(f"\nTop 10 users by login count:")
print(user_logins.head(n))

# Count logins and logoffs per user using crosstab
activity_counts = pd.crosstab(df['user'], df['activity'], margins=False)
activity_counts = activity_counts.reindex(columns=['Logon', 'Logoff'], fill_value=0)
activity_counts.columns = ['login_count', 'logoff_count']
activity_counts['in_out_difference'] = activity_counts['login_count'] - activity_counts['logoff_count']
activity_counts = activity_counts.sort_values('in_out_difference', ascending=False).reset_index()

print(f"\nUsers with highest login/logout differences:")
print(activity_counts.head(n))


Top 10 users by login count:
      user  login_count
0  AJF0370         1777
1  CBB0365         1758
2  WPR0368         1735
3  BAL0044         1709
4  IBB0359         1676
5  LBC0356         1662
6  EIS0041         1661
7  HDS0367         1655
8  DMG0358         1655
9  WDD0366         1450

Users with highest login/logout differences:
      user  login_count  logoff_count  in_out_difference
0  RZC0746          925           528                397
1  IKP0472         1016           621                395
2  HAD0246         1047           656                391
3  NOB0181          961           570                391
4  CTR0341          996           608                388
5  LAP0338          874           488                386
6  JDC0030          894           512                382
7  IRM0931         1063           684                379
8  ATE0869          903           524                379
9  HDD0806          903           525                378


In [ ]:

# Convert date to datetime using pd.to_datetime
df['datetime'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M:%S')
df['hour'] = df['datetime'].dt.hour

# Check if access occurred outside work hours (7 AM to 7 PM)
df['outside_work_hours'] = (df['hour'] < 7) | (df['hour'] >= 19)
# df['outside_work_hours_date'] = df['date'].where(df['outside_work_hours'], '-')

# Join with activity counts
df_extended = df.merge(activity_counts, on='user', how='left')

# Filter and analyze after-hours access
after_hours_summary = df_extended.groupby('user').agg({
    'outside_work_hours': ['sum', 'count'],
    'activity': 'count'
}).reset_index()

after_hours_summary.columns = ['user', 'after_hours_count', 'total_after_hours_records', 'total_records']
# after_hours_summary.columns = ['user', 'after_hours_count', 'total_records']
after_hours_summary['after_hours_percentage'] = (after_hours_summary['after_hours_count'] /
                                                after_hours_summary['total_records'] * 100)
after_hours_summary = after_hours_summary.sort_values('after_hours_count', ascending=False)

print(f"\nAfter-hours access summary (top 10 users):")
print(after_hours_summary.head(10))

# Show connect events outside work hours
connect_after_hours = df_extended[(df_extended['activity'] == 'Logon') &
                                 (df_extended['outside_work_hours'] == True)]
print(f"\nConnect events outside work hours: {len(connect_after_hours)}")
if len(connect_after_hours) > 0:
    print("Sample after-hours connections:")
    print(connect_after_hours[['user', 'pc', 'date', 'hour']].head(10))


After-hours access summary (top 10 users):
        user  after_hours_count  total_after_hours_records  total_records  \
969  WPR0368               1898                       3470           3470   
34   AJF0370               1695                       3267           3267   
937  UIR0043               1641                       2860           2860   
577  KRV0372               1637                       2820           2820   
674  MNS0224               1632                       2826           2826   
157  CBB0365               1627                       3256           3256   
554  KJP0371               1613                       2788           2788   
924  TSM0363               1612                       2794           2794   
399  HDS0367               1612                       3035           3035   
839  SAK0375               1611                       2828           2828   

     after_hours_percentage  
969               54.697406  
34                51.882461  
937               

In [ ]:

# Use groupby with multiple aggregations
pc_access_analysis = df.groupby(['user', 'pc']).agg({
    'id': 'count',  # access_count
    'activity': 'count'  # redundant but keeping for consistency
}).reset_index()
pc_access_analysis.columns = ['user', 'pc', 'access_count', 'activity_count']

# Count unique users per PC and unique PCs per user using nunique
pc_user_stats = df.groupby('pc')['user'].nunique().reset_index()
pc_user_stats.columns = ['pc', 'unique_user_per_pc_count']

user_pc_stats = df.groupby('user')['pc'].nunique().reset_index()
user_pc_stats.columns = ['user', 'unique_pc_per_user']

# Filter users with reasonable PC access
user_pc_stats_filtered = user_pc_stats[user_pc_stats['unique_pc_per_user'] > 0] #<500

# Merge all statistics
pc_access_final = pc_access_analysis.merge(user_pc_stats_filtered, on='user', how='inner')
pc_access_final = pc_access_final.merge(pc_user_stats, on='pc', how='left')

# Select and reorder columns
pc_access_final = pc_access_final[['pc', 'user', 'access_count', 'unique_user_per_pc_count', 'unique_pc_per_user']]
pc_access_final = pc_access_final.sort_values(['unique_user_per_pc_count', 'pc', 'user'], ascending=[False, True, True])

print(f"\nPC access analysis (top 20):")
print(pc_access_final.sort_values(by='unique_pc_per_user', ascending=False).head(20))



PC access analysis (top 20):
            pc     user  access_count  unique_user_per_pc_count  \
23772  PC-4524  WPR0368             2                        27   
23715  PC-3651  WPR0368             4                        23   
23877  PC-5952  WPR0368             4                        34   
24167  PC-9822  WPR0368             2                        20   
24025  PC-7865  WPR0368             4                        30   
24075  PC-8541  WPR0368             4                        32   
23468  PC-0094  WPR0368             2                        28   
23819  PC-5102  WPR0368             2                        25   
23653  PC-2684  WPR0368             2                        23   
23619  PC-2139  WPR0368             4                        27   
23796  PC-4753  WPR0368             6                        22   
23795  PC-4748  WPR0368             2                        22   
23491  PC-0262  WPR0368             6                        24   
23655  PC-2697  WPR0368         

In [ ]:
# Search for specific PCs and users using isin()
pc_list = ["PC-0843"]
user_list = ["IIW0249"]

specific_pcs = pc_access_final[pc_access_final['pc'].isin(pc_list)]
specific_users = pc_access_final[pc_access_final['user'].isin(user_list)]

if len(specific_pcs) > 0:
    print(f"\nSpecific PCs analysis:")
    print(specific_pcs)

if len(specific_users) > 0:
    print(f"\nSpecific users analysis:")
    print(specific_users)


Specific PCs analysis:
            pc     user  access_count  unique_user_per_pc_count  \
78     PC-0843  ACC0042             4                        28   
780    PC-0843  AJF0370             4                        28   
1595   PC-0843  BAL0044             4                        28   
2210   PC-0843  BBS0039             2                        28   
2675   PC-0843  BKW0374             2                        28   
3311   PC-0843  BSS0369             4                        28   
4563   PC-0843  CCA0046             8                        28   
5154   PC-0843  CSC0217             2                        28   
6184   PC-0843  DMG0358             2                        28   
6861   PC-0843  EIS0041             4                        28   
7626   PC-0843  GTD0219             2                        28   
8028   PC-0843  HDS0367             8                        28   
8725   PC-0843  HWY0038             2                        28   
9059   PC-0843  IBB0359             4 

In [ ]:

user_summary = df.groupby('user').agg({
    'pc': ['nunique', 'count'],  # unique PCs and total access count
    'activity': lambda x: list(x.unique()),  # list of activities
    'outside_work_hours': ['sum', 'any'],  # count and flag for after-hours
    'datetime': ['min', 'max']  # first and last access
}).reset_index()

# Flatten column names
user_summary.columns = ['user', 'unique_pc_count', 'total_access_count', 'activity_types',
                       'after_hours_count', 'has_after_hours', 'first_access', 'last_access']

# Add PC lists using groupby and apply
pc_lists = df.groupby('user')['pc'].apply(lambda x: sorted(list(x.unique()))).reset_index()
pc_lists.columns = ['user', 'pc_list']
user_summary = user_summary.merge(pc_lists, on='user')

# Sort by unique PC count
user_summary = user_summary.sort_values('unique_pc_count', ascending=False)

print(f"\nComprehensive User Analysis (top 15):")
display_cols = ['user', 'unique_pc_count', 'total_access_count', 'after_hours_count', 'has_after_hours']
print(user_summary[display_cols].head(15))

# Find users with access to all PCs
max_pc_count = df['pc'].nunique()
users_with_full_access = user_summary[user_summary['unique_pc_count'] == max_pc_count]

print(f"\nDataset Summary:")
print(f"Total unique PCs: {max_pc_count}")
print(f"Users with access to all PCs: {len(users_with_full_access)}")

if len(users_with_full_access) > 0:
    print("Users with full PC access:")
    print(users_with_full_access[['user', 'unique_pc_count', 'total_access_count']])




Comprehensive User Analysis (top 15):
        user  unique_pc_count  total_access_count  after_hours_count  \
969  WPR0368              720                3470               1898   
34   AJF0370              661                3267               1695   
674  MNS0224              659                2826               1632   
544  KCF0045              640                2780               1538   
479  JCC0225              639                2750               1562   
783  QNP0216              639                2814               1582   
589  LBC0356              636                3053               1547   
526  JRS0362              636                2776               1592   
554  KJP0371              635                2788               1613   
157  CBB0365              634                3256               1627   
267  DMG0358              633                3038               1599   
937  UIR0043              631                2860               1641   
399  HDS0367             

In [ ]:

print(f"\n=== FINAL SUMMARY ===")
print(f"Total records: {len(df):,}")
print(f"Unique users: {df['user'].nunique():,}")
print(f"Unique PCs: {df['pc'].nunique():,}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"Connect events: {(df['activity'] == 'Logon').sum():,}")
print(f"Disconnect events: {(df['activity'] == 'Logoff').sum():,}")
print(f"After-hours events: {df['outside_work_hours'].sum():,}")
print(f"Users with after-hours access: {user_summary['has_after_hours'].sum():,}")

# Activity distribution
activity_dist = df['activity'].value_counts()
print(f"\nActivity Distribution:")
for activity, count in activity_dist.items():
    print(f"  {activity}: {count:,} ({count/len(df)*100:.1f}%)")

# Time-based analysis
hourly_activity = df.groupby(df['datetime'].dt.hour).size()
peak_hour = hourly_activity.idxmax()
print(f"\nPeak activity hour: {peak_hour}:00 with {hourly_activity[peak_hour]:,} events")


=== FINAL SUMMARY ===
Total records: 854,859
Unique users: 1,000
Unique PCs: 1,003
Date range: 2010-01-02 06:49:00 to 2011-05-17 06:43:35
Connect events: 470,591
Disconnect events: 384,268
After-hours events: 109,607
Users with after-hours access: 319

Activity Distribution:
  Logon: 470,591 (55.0%)
  Logoff: 384,268 (45.0%)

Peak activity hour: 7:00 with 149,697 events


In [ ]:
print("=== USER-PC ACCESS ANALYSIS ===\n")

# ALL USERS WHO HAVE ACCESS TO EACH PC
print("1. ALL USERS WHO HAVE ACCESS TO EACH PC")
print("="*80)

# Group by PC and get list of all users who accessed it
pc_to_users = df.groupby('pc').agg({
    'user': lambda x: sorted(list(set(x))),  # Unique users, sorted
    'id': 'count'  # Total access count for this PC
}).reset_index()

pc_to_users.columns = ['pc', 'users_with_access', 'total_accesses']
pc_to_users['unique_user_count'] = pc_to_users['users_with_access'].apply(len)

# Sort by number of users (most shared PCs first)
pc_to_users = pc_to_users.sort_values('unique_user_count', ascending=False)

print(f"Total PCs in system: {len(pc_to_users)}")
print(f"Most shared PC has {pc_to_users['unique_user_count'].max()} different users")
print(f"Least shared PC has {pc_to_users['unique_user_count'].min()} different users")
print("\nTop 10 most shared PCs:")

for idx, row in pc_to_users.head(10).iterrows():
    print(f"\nPC: {row['pc']}")
    print(f"  Users with access ({row['unique_user_count']}): {', '.join(row['users_with_access'][:10])}")
    if len(row['users_with_access']) > 10:
        print(f"   ... and {len(row['users_with_access']) - 10} more users")
    print(f" Total accesses: {row['total_accesses']}")

print("\n" + "="*80 + "\n")

=== USER-PC ACCESS ANALYSIS ===

1. ALL USERS WHO HAVE ACCESS TO EACH PC
Total PCs in system: 1003
Most shared PC has 264 different users
Least shared PC has 1 different users

Top 10 most shared PCs:

PC: PC-4124
  Users with access (264): ABM0845, ACV0812, AHC0142, AHD0848, AHG0634, AHM0410, AIB0948, AIP0982, AJD0074, AJN0607
   ... and 254 more users
 Total accesses: 25514

PC: PC-3847
  Users with access (137): AHC0142, AHM0410, AIB0948, ANT0430, AOK0844, AOS0987, ARD0509, ATE0869, BAH0875, BAL0044
   ... and 127 more users
 Total accesses: 4264

PC: PC-7965
  Users with access (37): ACC0042, AJF0370, BAL0044, BJP0134, BKW0374, BSS0369, CBB0365, CCA0046, DFR0218, DMG0358
   ... and 27 more users
 Total accesses: 794

PC: PC-4722
  Users with access (36): ACC0042, AKR0057, BBS0039, BKW0374, BTL0226, CBB0365, CCA0046, CSC0217, DFR0218, DMG0358
   ... and 26 more users
 Total accesses: 802

PC: PC-0692
  Users with access (36): AJF0370, BAL0044, BBS0039, BKW0374, BSS0369, CBB0365, CCA

In [ ]:
# ALL PCs THAT EACH USER TOUCHES
print("2. ALL PCs THAT EACH USER TOUCHES")
print("="*80)

# Group by User and get list of all PCs they accessed
user_to_pcs = df.groupby('user').agg({
    'pc': lambda x: sorted(list(set(x))),  # Unique PCs, sorted
    'id': 'count'  # Total access count for this user
}).reset_index()

user_to_pcs.columns = ['user', 'pcs_accessed', 'total_accesses']
user_to_pcs['unique_pc_count'] = user_to_pcs['pcs_accessed'].apply(len)

# Sort by number of PCs accessed (users with broadest access first)
user_to_pcs = user_to_pcs.sort_values('unique_pc_count', ascending=False)

print(f"Total users in system: {len(user_to_pcs)}")
print(f"User with broadest access touches {user_to_pcs['unique_pc_count'].max()} different PCs")
print(f"User with narrowest access touches {user_to_pcs['unique_pc_count'].min()} different PCs")
print("\nTop 10 users with broadest PC access:")

for idx, row in user_to_pcs.head(10).iterrows():
    print(f"\nUser: {row['user']}")
    print(f"PCs accessed ({row['unique_pc_count']}): {', '.join(row['pcs_accessed'][:10])}")
    if len(row['pcs_accessed']) > 10:
        print(f"   ... and {len(row['pcs_accessed']) - 10} more PCs")
    print(f"Total accesses: {row['total_accesses']}")

print("\n" + "="*80 + "\n")

2. ALL PCs THAT EACH USER TOUCHES
Total users in system: 1000
User with broadest access touches 720 different PCs
User with narrowest access touches 1 different PCs

Top 10 users with broadest PC access:

User: WPR0368
PCs accessed (720): PC-0008, PC-0039, PC-0072, PC-0092, PC-0094, PC-0115, PC-0118, PC-0120, PC-0132, PC-0133
   ... and 710 more PCs
Total accesses: 3470

User: AJF0370
PCs accessed (661): PC-0004, PC-0008, PC-0039, PC-0044, PC-0072, PC-0092, PC-0115, PC-0120, PC-0133, PC-0141
   ... and 651 more PCs
Total accesses: 3267

User: MNS0224
PCs accessed (659): PC-0004, PC-0008, PC-0039, PC-0044, PC-0115, PC-0118, PC-0120, PC-0133, PC-0141, PC-0148
   ... and 649 more PCs
Total accesses: 2826

User: KCF0045
PCs accessed (640): PC-0004, PC-0008, PC-0039, PC-0044, PC-0072, PC-0092, PC-0118, PC-0120, PC-0132, PC-0133
   ... and 630 more PCs
Total accesses: 2780

User: JCC0225
PCs accessed (639): PC-0004, PC-0008, PC-0072, PC-0092, PC-0132, PC-0133, PC-0146, PC-0148, PC-0151, PC-0

In [ ]:
# SEARCH FUNCTIONS
def find_users_for_pc(pc_name):
    """Find all users who have accessed a specific PC"""
    result = pc_to_users[pc_to_users['pc'] == pc_name]
    if len(result) == 0:
        print(f"PC '{pc_name}' not found in the system")
        return None

    row = result.iloc[0]
    print(f"PC: {pc_name}")
    print(f"Users with access ({len(row['users_with_access'])}):")
    for i, user in enumerate(row['users_with_access'], 1):
        print(f"   {i:2d}. {user}")
    print(f"Total accesses: {row['total_accesses']}")
    return row['users_with_access']

def find_pcs_for_user(username):
    """Find all PCs that a specific user has accessed"""
    result = user_to_pcs[user_to_pcs['user'] == username]
    if len(result) == 0:
        print(f"User '{username}' not found in the system")
        return None

    row = result.iloc[0]
    print(f"User: {username}")
    print(f"PCs accessed ({len(row['pcs_accessed'])}):")
    for i, pc in enumerate(row['pcs_accessed'], 1):
        print(f"   {i:2d}. {pc}")
    print(f"Total accesses: {row['total_accesses']}")
    return row['pcs_accessed']

In [ ]:
# EXAMPLE SEARCHES
print("3. EXAMPLE SEARCHES")
print("="*50)

# Example: Search for specific PCs
example_pcs = ["PC-3471", "PC-7117"]######################################

print("Searching for specific PCs:")
for pc in example_pcs:
    print(f"\n--- Results for {pc} ---")
    find_users_for_pc(pc)

print("\n" + "-"*80)

# Example: Search for specific users
example_users = ["CJM0273", "RAA1455"]#########################################

print("Searching for specific users:")
for user in example_users:
    print(f"\n--- Results for {user} ---")
    find_pcs_for_user(user)

print("\n" + "="*80 + "\n")

3. EXAMPLE SEARCHES
Searching for specific PCs:

--- Results for PC-3471 ---
PC 'PC-3471' not found in the system

--- Results for PC-7117 ---
PC 'PC-7117' not found in the system

--------------------------------------------------------------------------------
Searching for specific users:

--- Results for CJM0273 ---
User 'CJM0273' not found in the system

--- Results for RAA1455 ---
User 'RAA1455' not found in the system




In [ ]:
# SECURITY ANALYSIS
print("4. SECURITY ANALYSIS")
print("="*50)

# Find potentially risky PCs (accessed by many users)
risky_pcs = pc_to_users[pc_to_users['unique_user_count'] >= 10]
print(f"Potentially risky PCs (≥10 users): {len(risky_pcs)}")
if len(risky_pcs) > 0:
    print("Top 5 most shared PCs:")
    for idx, row in risky_pcs.head(5).iterrows():
        print(f"   {row['pc']}: {row['unique_user_count']} users, {row['total_accesses']} total accesses")

print()

# Find potentially privileged users (access to many PCs)
privileged_users = user_to_pcs[user_to_pcs['unique_pc_count'] >= 10]
print(f"Potentially privileged users (≥10 PCs): {len(privileged_users)}")
if len(privileged_users) > 0:
    print("Top 5 users with broadest access:")
    for idx, row in privileged_users.head(5).iterrows():
        print(f"   {row['user']}: {row['unique_pc_count']} PCs, {row['total_accesses']} total accesses")

print()

# Users with access to only 1 PC (potentially dedicated/restricted users)
restricted_users = user_to_pcs[user_to_pcs['unique_pc_count'] == 1]
print(f"Restricted users (only 1 PC): {len(restricted_users)}")
if len(restricted_users) > 0:
    for idx, row in restricted_users.head(5).iterrows():
        print(f"   {row['user']}: {row['unique_pc_count']} PC, {row['total_accesses']} total accesses")

# PCs accessed by only 1 user (potentially personal/dedicated PCs)
personal_pcs = pc_to_users[pc_to_users['unique_user_count'] == 1]
print(f"Personal/dedicated PCs (only 1 user): {len(personal_pcs)}")
if len(personal_pcs) > 0:
    for idx, row in personal_pcs.head(5).iterrows():
        print(f"   {row['pc']}: {row['unique_user_count']} user, {row['total_accesses']} total accesses")

print("\n" + "="*80 + "\n")

4. SECURITY ANALYSIS
Potentially risky PCs (≥10 users): 943
Top 5 most shared PCs:
   PC-4124: 264 users, 25514 total accesses
   PC-3847: 137 users, 4264 total accesses
   PC-7965: 37 users, 794 total accesses
   PC-4722: 36 users, 802 total accesses
   PC-0692: 36 users, 800 total accesses

Potentially privileged users (≥10 PCs): 85
Top 5 users with broadest access:
   WPR0368: 720 PCs, 3470 total accesses
   AJF0370: 661 PCs, 3267 total accesses
   MNS0224: 659 PCs, 2826 total accesses
   KCF0045: 640 PCs, 2780 total accesses
   JCC0225: 639 PCs, 2750 total accesses

Restricted users (only 1 PC): 680
   LNR0656: 1 PC, 692 total accesses
   SOG0411: 1 PC, 692 total accesses
   MLL0598: 1 PC, 692 total accesses
   SBN0971: 1 PC, 898 total accesses
   MLP0668: 1 PC, 692 total accesses
Personal/dedicated PCs (only 1 user): 40
   PC-5186: 1 user, 692 total accesses
   PC-5179: 1 user, 1014 total accesses
   PC-9947: 1 user, 692 total accesses
   PC-0253: 1 user, 692 total accesses
   PC-

In [ ]:
# SUMMARY STATISTICS
print("5. SUMMARY STATISTICS")
print("="*50)

print("PC Access Distribution:")
pc_stats = pc_to_users['unique_user_count'].describe()
print(f"   Mean users per PC: {pc_stats['mean']:.1f}")
print(f"   Median users per PC: {pc_stats['50%']:.1f}")
print(f"   Most shared PC: {pc_stats['max']:.0f} users")
print(f"   Least shared PC: {pc_stats['min']:.0f} user(s)")

print("\nUser Access Distribution:")
user_stats = user_to_pcs['unique_pc_count'].describe()
print(f"   Mean PCs per user: {user_stats['mean']:.1f}")
print(f"   Median PCs per user: {user_stats['50%']:.1f}")
print(f"   User with broadest access: {user_stats['max']:.0f} PCs")
print(f"   User with narrowest access: {user_stats['min']:.0f} PC(s)")

5. SUMMARY STATISTICS
PC Access Distribution:
   Mean users per PC: 24.2
   Median users per PC: 25.0
   Most shared PC: 264 users
   Least shared PC: 1 user(s)

User Access Distribution:
   Mean PCs per user: 24.3
   Median PCs per user: 1.0
   User with broadest access: 720 PCs
   User with narrowest access: 1 PC(s)


In [ ]:
print("=== USER-PC ACCESS ANALYSIS ===\n")

# === 1. ALL USERS WHO HAVE ACCESS TO EACH PC ===
print("1. ALL USERS WHO HAVE ACCESS TO EACH PC")
print("="*50)

# Group by PC and get list of all users who accessed it
pc_to_users = df.groupby('pc').agg({
    'user': lambda x: sorted(list(set(x))),  # Unique users, sorted
    'id': 'count'  # Total access count for this PC
}).reset_index()

pc_to_users.columns = ['pc', 'users_with_access', 'total_accesses']
pc_to_users['unique_user_count'] = pc_to_users['users_with_access'].apply(len)

# Filter out PCs accessed by only one user
pc_to_users = pc_to_users[pc_to_users['unique_user_count'] > 1]

# Sort by number of users (most shared PCs first)
pc_to_users = pc_to_users.sort_values('unique_user_count', ascending=False)

print(f"PCs with multiple users: {len(pc_to_users)}")
print(f"Most shared PC has {pc_to_users['unique_user_count'].max()} different users")
print(f"Least shared PC has {pc_to_users['unique_user_count'].min()} different users")

# === 2. ALL PCs THAT EACH USER TOUCHES ===
print("2. ALL PCs THAT EACH USER TOUCHES")
print("="*50)

# Group by User and get list of all PCs they accessed
user_to_pcs = df.groupby('user').agg({
    'pc': lambda x: sorted(list(set(x))),  # Unique PCs, sorted
    'id': 'count'  # Total access count for this user
}).reset_index()

user_to_pcs.columns = ['user', 'pcs_accessed', 'total_accesses']
user_to_pcs['unique_pc_count'] = user_to_pcs['pcs_accessed'].apply(len)

# Filter out users who access only one PC
user_to_pcs = user_to_pcs[user_to_pcs['unique_pc_count'] > 1]

# Sort by number of PCs accessed (users with broadest access first)
user_to_pcs = user_to_pcs.sort_values('unique_pc_count', ascending=False)

print(f"Users with access to multiple PCs: {len(user_to_pcs)}")
print(f"User with broadest access touches {user_to_pcs['unique_pc_count'].max()} different PCs")
print(f"User with narrowest access touches {user_to_pcs['unique_pc_count'].min()} different PCs")


=== USER-PC ACCESS ANALYSIS ===

1. ALL USERS WHO HAVE ACCESS TO EACH PC
PCs with multiple users: 963
Most shared PC has 264 different users
Least shared PC has 2 different users
2. ALL PCs THAT EACH USER TOUCHES
Users with access to multiple PCs: 320
User with broadest access touches 720 different PCs
User with narrowest access touches 2 different PCs


In [ ]:
users_with_after_hours = user_summary[user_summary['has_after_hours'] == True]['user'].unique()

user_to_pcs['has_after_hours'] = user_to_pcs['user'].isin(users_with_after_hours)
# pc_to_users['has_after_hours'] = pc_to_users['user'].isin(pc_to_users)


psychometric_data = pd.read_csv('/psychometric.csv')
# user_to_pcs['O'] = user_to_pcs['user'].isin(psychometric_data['user_id'])
user_to_pcs = user_to_pcs.merge(
    psychometric_data[['user_id', 'O', 'C', 'E', 'A', 'N']],
    left_on='user',
    right_on='user_id',
    how='left'
)
user_to_pcs['OCEAN'] = user_to_pcs['O'] + user_to_pcs['C'] + user_to_pcs['E'] + user_to_pcs['A'] + user_to_pcs['N']
user_to_pcs.drop(columns=['user_id', 'O', 'C', 'E', 'A', 'N'], inplace=True)

In [ ]:
def categorize(value, limit):
    if value < limit:
        return 'Low'
    elif value == limit:
        return 'Medium'
    else:
        return 'High'

# def categorize(value, limit):
#     return 'Low' if value < limit else 'Medium' if value == limit else 'High'

# Apply to each column
user_to_pcs['OCEAN_level'] = user_to_pcs['OCEAN'].apply(lambda x: categorize(x, 125))
# user_to_pcs['O'] = user_to_pcs['O'].apply(lambda x: categorize(x, 25))
# user_to_pcs['C'] = user_to_pcs['C'].apply(lambda x: categorize(x, 25))
# user_to_pcs['E'] = user_to_pcs['E'].apply(lambda x: categorize(x, 25))
# user_to_pcs['A'] = user_to_pcs['A'].apply(lambda x: categorize(x, 25))
# user_to_pcs['N'] = user_to_pcs['N'].apply(lambda x: categorize(x, 25))

In [ ]:
user_to_pcs.head()

,user,pcs_accessed,total_accesses,unique_pc_count,has_after_hours,OCEAN,OCEAN_level
0,WPR0368,"[PC-0008, PC-0039, PC-0072, PC-0092, PC-0094, ...",3470,720,True,170,High
1,AJF0370,"[PC-0004, PC-0008, PC-0039, PC-0044, PC-0072, ...",3267,661,True,132,High
2,MNS0224,"[PC-0004, PC-0008, PC-0039, PC-0044, PC-0115, ...",2826,659,True,164,High
3,KCF0045,"[PC-0004, PC-0008, PC-0039, PC-0044, PC-0072, ...",2780,640,True,153,High
4,JCC0225,"[PC-0004, PC-0008, PC-0072, PC-0092, PC-0132, ...",2750,639,True,165,High


In [ ]:
user_to_pcs.to_csv("logon_user_to_pcs.csv", index=False)

In [ ]:
pc_to_users.head()

,pc,users_with_access,total_accesses,unique_user_count
410,PC-4124,"[ABM0845, ACV0812, AHC0142, AHD0848, AHG0634, ...",25514,264
384,PC-3847,"[AHC0142, AHM0410, AIB0948, ANT0430, AOK0844, ...",4264,137
788,PC-7965,"[ACC0042, AJF0370, BAL0044, BJP0134, BKW0374, ...",794,37
134,PC-1289,"[ACC0042, AJF0370, BAL0044, BBS0039, BSS0369, ...",1075,36
463,PC-4722,"[ACC0042, AKR0057, BBS0039, BKW0374, BTL0226, ...",802,36


In [ ]:
pc_to_users.to_csv("logon_pc_to_users.csv", index=False)

In [ ]:
# 320 users with access to a total of 963 PCs
# 320 out of 1 000 users and 963 out of 1 003 PCs
# Some pcs accessed outside of work hours
######################################################################